In [ ]:
## Поиск генов, связанных с метастазированием при светлоклеточном раке почки

In [ ]:
##Устаеавливаем необходимые пакеты. 
pip install scanpy

In [ ]:
pip install decoupler

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import decoupler as dc
from anndata import AnnData

In [5]:
## Загружаем данные TCGA.
data = pd.read_csv('C:\\Users\\с\\Downloads\\TCGA.KIRC.sampleMap_HiSeqV2\\HiSeqV2',  sep='\t')

## Усатнавливаем колонку с генами в качестве индексов.
data = data.set_index('sample')
data

,TCGA-BP-4162-01,TCGA-CJ-5677-11,TCGA-DV-5566-01,TCGA-BP-5191-01,TCGA-BP-5200-01,TCGA-BP-4347-01,TCGA-BP-4770-01,TCGA-B0-5696-11,TCGA-BP-4762-01,TCGA-BP-4158-01,...,TCGA-B0-5104-01,TCGA-A3-3313-01,TCGA-B2-5633-01,TCGA-CJ-4872-01,TCGA-CJ-5684-01,TCGA-CJ-4886-01,TCGA-BP-5009-01,TCGA-BP-4969-01,TCGA-CZ-5461-01,TCGA-CW-5587-01
sample,,,,,,,,,,,,,,,,,,,,,
ARHGEF10L,10.5030,10.8969,10.7612,10.2063,10.0616,10.0193,8.4364,11.1427,9.9150,10.7621,...,10.5586,9.5556,10.2652,9.4981,10.2751,9.9525,10.1273,10.8817,10.1949,10.3294
HIF3A,5.5283,6.4943,5.7842,5.0063,5.3326,7.6102,6.0422,5.9789,3.7802,7.2039,...,6.2061,4.8253,5.1680,8.6195,6.2080,6.3116,6.7446,7.8293,6.1788,6.2293
RNF17,3.8036,0.0000,0.0000,0.0000,0.0000,0.3386,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.6128,0.0000
RNF10,11.4379,12.2130,11.5478,12.3439,11.5149,11.4065,12.1963,12.3946,11.3734,11.3251,...,11.7338,11.7709,11.4968,11.9561,11.7518,11.5698,12.1565,11.3048,11.5168,11.3489
RNF11,11.4180,11.8248,11.3190,10.3413,11.2923,10.9971,11.8936,11.7217,12.2329,11.2993,...,11.7629,10.2373,11.4637,11.4391,11.3992,11.4937,11.4015,10.8842,11.6230,11.5469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PTRF,12.7871,12.0428,13.1766,13.1159,13.6014,13.0401,15.4372,11.7112,14.1524,12.8333,...,12.9562,11.0444,13.3909,13.4411,13.3214,13.3236,13.5652,12.7908,13.1986,12.5529
BCL6B,9.8322,8.5567,10.3382,8.3105,10.9764,10.4252,9.2090,8.2285,10.5512,10.1991,...,9.9155,8.7818,10.5225,10.8029,10.4498,10.2815,10.1857,9.7342,10.1779,8.9658
GSTK1,11.6227,11.6819,11.8431,12.5657,11.1899,11.5918,10.8875,11.8056,11.3034,12.1317,...,11.4145,11.1925,10.3842,11.5255,11.9294,11.3444,11.9962,12.6813,11.5499,11.7339


In [175]:
## Загружаем клинические характеристики образцов.
metadata = pd.read_csv('C:\\Users\\с\\Downloads\\TCGA.KIRC.txt',  sep='\t')

## Усатнавливаем колонку с образцами качестве индексов.
metadata = metadata.set_index('sampleID')

metadata

In [174]:
## Посмотрим, какие есть клинические данные об образцах.
metadata.columns.tolist()

In [26]:
# Посмотрим, какая информация есть о метастазах.
metadata['pathologic_M'].unique()

array(['M0', 'MX', 'M1', nan], dtype=object)

In [164]:
# Посчитаем, сколько образцов с наличием и отсутствием метастазов содержится в датасете.

metadata['pathologic_M'].tolist().count('M1')
sns.barplot(data=pd.DataFrame({'M0': [metadata['pathologic_M'].tolist().count('M0')], 
                               'M1': [metadata['pathologic_M'].tolist().count('M1')],
                               
                              }), 
           )
plt.title('Number of samples by metastasis status')
plt.ylabel('Number of samples', fontsize=12)
plt.xlabel('pathologic_M', fontsize=12)

In [173]:
# Создадим объект Anndata. 
adata = AnnData(data.T)
adata.var_names_make_unique()
adata

In [103]:
# Добавим информацию о наличии или отсутствии метастазов. 
adata.obs['condition'] = 0
for i in data.columns:
    if metadata.loc[i]['pathologic_M'] == 'M1':
        adata.obs['condition'].loc[i] = 'Metastatiс'
    elif metadata.loc[i]['pathologic_M'] == 'M0':
        adata.obs['condition'].loc[i] = 'Non_metastatic'

In [165]:
# Добавим информацию об образцах.

adata.obs['sample_id'] = adata.obs.index.tolist()

# Посмотрим на таблицу с метаданными
adata.obs

In [166]:
# Для дальнейшего анализа оставим только образцы с наличием информации о метастазах
adata = adata[adata.obs.condition != 0]

adata.obs

In [99]:
# Оставим гены, с экспрессией больше 4 
dc.mask_features(adata, log=True, thr=4)

In [77]:
# Удалим гены с не изменяющейся экспрессией по образцам.
sc.pp.filter_genes(adata, min_cells=adata.shape[0])

In [95]:
# Нормализуем данные Log-CPM counts
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

In [178]:
# Посчитаем гены с дифферециальной экспрессией 
logFCs, pvals = dc.get_contrast(adata,
                                group_col=None,
                                condition_col='condition',
                                condition='Metastatiс',
                                reference='Non_metastatic',
                                method='t-test'
                               )
logFCs

In [168]:
## Постороим Волкано плот.
dc.plot_volcano(logFCs, pvals, 'Metastatiс.vs.Non_metastatic', top=15, sign_thr=0.05, lFCs_thr=0.5, lFCs_limit=10)

In [169]:
# Retrieve PROGENy model weights
progeny = dc.get_progeny(top=300)

# Infer pathway activities with consensus
pathway_acts, pathway_pvals = dc.run_consensus(mat=logFCs, net=progeny)
pathway_acts

In [170]:
# Посмотрим не гены с наиболее значимыми изменениями экспрессии

top_genes = dc.get_top_targets(logFCs, pvals, 'Metastatiс.vs.Non_metastatic', sign_thr=0.05, lFCs_thr=0.5)
top_genes

In [172]:
for gene in top_genes.head(400)['name']:
    print(gene)